# 回帰木

- 決定木を用いた回帰

In [1]:
import pandas as pd
import seaborn as sns

# Iris データセット
iris = sns.load_dataset('iris')

# species のダミー変数化
df = pd.get_dummies(data=iris)
# 確認
df.iloc[[0, 50, 100]]

,sepal_length,sepal_width,petal_length,petal_width,species_setosa,species_versicolor,species_virginica
0,5.1,3.5,1.4,0.2,1,0,0
50,7.0,3.2,4.7,1.4,0,1,0
100,6.3,3.3,6.0,2.5,0,0,1


<hr>

![kNN](slides/8_37.png)

### 一番相関がある petal_length を目的変数として回帰木を作成

In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 説明変数、目的変数の設定
Y_label = 'petal_length'
X = df.drop([Y_label], axis=1)
Y = df[Y_label]

# 回帰木を作成
# - max_depth: 最大深さ
model = DecisionTreeRegressor(max_depth=5)
# 学習
model.fit(X, Y)

# MSE: 平均二乗誤差
# R^2: 決定係数
Y_predict = model.predict(X)
print('MSE={:.3f}, R^2={:.3f}'.format(
    mean_squared_error(Y, Y_predict), r2_score(Y, Y_predict)))

MSE=0.034, R^2=0.989


<hr>

### 回帰木の精度の比較

目的変数

1. petal_length: 他の変数とかなり強く相関
2. sepal_width: 他の変数との相関が弱い

In [3]:
# 目的変数のリスト
Y_labels_list = ['petal_length', 'sepal_width']

# 回帰木を作成
model = DecisionTreeRegressor(max_depth=5)

for i in range(0, len(Y_labels_list)):
    # 説明変数、目的変数の設定
    Y_label = Y_labels_list[i]
    X = df.drop([Y_label], axis=1)
    Y = df[Y_label]
    # 学習
    model.fit(X, Y)
    # 教師データについてモデルからの予測値を計算
    Y_predict = model.predict(X)

    # 精度
    print('{}: {}\tMSE={:.3f}, R^2={:.3f}'.format(i, Y_label,
        mean_squared_error(Y, Y_predict), r2_score(Y, Y_predict)))

0: petal_length	MSE=0.034, R^2=0.989
1: sepal_width	MSE=0.040, R^2=0.786


<hr>

### GridSearchCV を用いた交差検証法の実行

- 未知のデータに対する予測精度の検証

In [4]:
from sklearn.model_selection import GridSearchCV

# 説明変数、目的変数の設定
Y_label = 'petal_length'
X = df.drop([Y_label], axis=1)
Y = df[Y_label]

# 回帰木を作成
model = DecisionTreeRegressor()

# 探索パラメータ、グリッドの設定
param_range = list(range(1, 20, 1))
grid = [{'max_depth': param_range}]

# グリッドサーチの作成、実行
gs = GridSearchCV(model, param_grid=grid, scoring='r2', cv=10, n_jobs=-1)
gs.fit(X, Y)

# 結果の表示
print('best R2={:.3f}'.format(gs.best_score_))
print('parameter:', gs.best_params_)

best R2=0.185
parameter: {'max_depth': 4}
